In [1]:
val moviesRdd =  sc.textFile("hdfs://localhost:9000/movies/movies.csv")
val ratingRdd = sc.textFile("hdfs://localhost:9000/ratings/ratings.csv")

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.80.128:4041
SparkContext available as 'sc' (version = 2.4.7, master = local[*], app id = local-1646252013108)
SparkSession available as 'spark'


moviesRdd: org.apache.spark.rdd.RDD[String] = hdfs://localhost:9000/movies/movies.csv MapPartitionsRDD[1] at textFile at <console>:25
ratingRdd: org.apache.spark.rdd.RDD[String] = hdfs://localhost:9000/ratings/ratings.csv MapPartitionsRDD[3] at textFile at <console>:26


In [2]:
moviesRdd.count()

res0: Long = 9743


In [4]:
ratingRdd.count()

res2: Long = 100837


In [5]:
moviesRdd.first()

res3: String = movieId,title,genres


In [7]:
ratingRdd.first()

res5: String = userId,movieId,rating,timestamp


In [10]:
val header = moviesRdd.first()
val moviesContentRdd = moviesRdd.filter (line => line != header)
moviesContentRdd.take(2)

header: String = movieId,title,genres
moviesContentRdd: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[4] at filter at <console>:27
res8: Array[String] = Array(1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy, 2,Jumanji (1995),Adventure|Children|Fantasy)


In [11]:
val rheader = ratingRdd.first()
val ratingsContentRdd = ratingRdd.filter (line => line != rheader)
ratingsContentRdd.take(2)

rheader: String = userId,movieId,rating,timestamp
ratingsContentRdd: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[5] at filter at <console>:27
res9: Array[String] = Array(1,1,4.0,964982703, 1,3,4.0,964981247)


In [25]:
// parse moviescsv without header input tuple
val moviesParsedRdd = moviesContentRdd
                        .map (line => line.trim())
                        .filter ( line => !line.isEmpty())
                        .map ( line => line.split(","))
moviesParsedRdd.take(4)

moviesParsedRdd: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[16] at map at <console>:32
res21: Array[Array[String]] = Array(Array(1, Toy Story (1995), Adventure|Animation|Children|Comedy|Fantasy), Array(2, Jumanji (1995), Adventure|Children|Fantasy), Array(3, Grumpier Old Men (1995), Comedy|Romance), Array(4, Waiting to Exhale (1995), Comedy|Drama|Romance))


In [34]:
// convert moviesParsedRdd into tuple of 3 elements 
// (INT, String, String), using _1, _2, _3
val moviesTupleRdd = moviesParsedRdd.map (arr => (arr(0).toInt, (arr(1), arr(2) ) ) )
moviesTupleRdd.take(4)

moviesTupleRdd: org.apache.spark.rdd.RDD[(Int, (String, String))] = MapPartitionsRDD[25] at map at <console>:30
res28: Array[(Int, (String, String))] = Array((1,(Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy)), (2,(Jumanji (1995),Adventure|Children|Fantasy)), (3,(Grumpier Old Men (1995),Comedy|Romance)), (4,(Waiting to Exhale (1995),Comedy|Drama|Romance)))


In [28]:
// FIXME: 
// using case class , domain object
// case class Movie(id: Int, title: String, genres: String)
// val moviesDataRdd = moviesTupleRdd.map ( t => Movie(t._1, t._2, t._3))
// moviesDataRdd.take(4)

org.apache.spark.SparkDriverExecutionException:  Execution error

In [29]:
val ratingParsedRdd = ratingsContentRdd
                        .map (line => line.trim())
                        .filter ( line => !line.isEmpty())
                        .map ( line => line.split(","))

ratingParsedRdd.take(4)

ratingParsedRdd: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[22] at map at <console>:29
res25: Array[Array[String]] = Array(Array(1, 1, 4.0, 964982703), Array(1, 3, 4.0, 964981247), Array(1, 6, 4.0, 964982224), Array(1, 47, 5.0, 964983815))


In [36]:
// get ratings in tuple format
// bring the movieId to first member
val ratingsTupleRdd = ratingParsedRdd.map (rating => (rating(1).toInt, //movieId
                                                      (rating(0).toInt, //  userId
                                                      rating(2).toDouble, //rating
                                                      rating(3).toLong ) // timestamp
                                                     ))
                                           
ratingsTupleRdd.take(4)

ratingsTupleRdd: org.apache.spark.rdd.RDD[(Int, (Int, Double, Long))] = MapPartitionsRDD[26] at map at <console>:30
res29: Array[(Int, (Int, Double, Long))] = Array((1,(1,4.0,964982703)), (3,(1,4.0,964981247)), (6,(1,4.0,964982224)), (47,(1,5.0,964983815)))


In [38]:
val outputRdd = moviesTupleRdd.join(ratingsTupleRdd)
outputRdd.take(3)

outputRdd: org.apache.spark.rdd.RDD[(Int, ((String, String), (Int, Double, Long)))] = MapPartitionsRDD[32] at join at <console>:28
res30: Array[(Int, ((String, String), (Int, Double, Long)))] = Array((1084,((Bonnie and Clyde (1967),Crime|Drama),(4,5.0,945079259))), (1084,((Bonnie and Clyde (1967),Crime|Drama),(6,3.0,845555454))), (1084,((Bonnie and Clyde (1967),Crime|Drama),(32,4.0,856737338))))
